In [9]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
from psycopg2 import sql


# PostgreSQL connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}
post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\AWIROBLABEL.IN645.2023022804100111'

# Define positions for slicing each line into tuples
lengths = [
    2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 9, 1, 30, 60, 6, 1, 7, 11, 8, 4, 1, 64
]

# Calculate the end positions based on the lengths
end_positions = [sum(lengths[:i + 1]) for i in range(len(lengths))]

# Create positions as tuples of start and end positions
positions = [(start, end) for start, end in zip([0] + end_positions, end_positions)]

print(positions)

# Open the file and read its content
with open(text_data_path, 'r', encoding='utf-8') as file:
    text_data = file.read()
def process_text_data(text_data, positions):
    # List to store extracted data as tuples
    extracted_data = []

    try:
        for line in text_data.split('\n'):
            try:
                # Slice each line based on positions and create a tuple
                line_data = tuple(line[start:end].strip() for start, end in positions)

                # Check if the tuple is not empty (contains at least one non-blank element)
                if any(field for field in line_data):
                    # Append the tuple to the list
                    extracted_data.append(line_data)

            except Exception as e:
                print(f"Error processing line: {line}. Exception: {e}")

    except Exception as e:
        print(f"Error processing text data. Error: {str(e)}")

    # Return the list of tuples
    return extracted_data


result = process_text_data(text_data, positions)
print(result)
current_datetimestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(current_datetimestamp)

# Assuming creation_date, last_update_date are datetime objects
# creation_date = datetime.now()
# last_update_date = datetime.now()
# Assuming created_by, last_updated_by, last_update_login are strings
# created_by = 'CSPOMS'
# last_updated_by = 'CSPOMS'
# last_update_login = 'CSPOMS'
# request id generation
nextval_request_id = f"select nextval('cspoms.XXPO_LABEL_FLAG_AWI_TRICEPS_STG_S')"
post_cur.execute(nextval_request_id)
request_id = post_cur.fetchone()[0]
# post_conn.commit()
print(f"Request ID: {request_id}")


def insert_data_to_table(extracted_data, table_name, post_cur, column_names,request_id):
    # Create the INSERT query with explicit column names and placeholders
    insert_query = f"INSERT INTO {table_name} (request_id, {', '.join(column_names)}, creation_date, last_update_date, created_by, last_updated_by, last_update_login) VALUES (%s, {', '.join(['%s']*len(column_names))}, %s, %s, %s, %s, %s)"
    print(insert_query)

    try:
        # Execute the INSERT query with the data
        post_cur.executemany(insert_query, [(request_id,) + row + (current_datetimestamp, current_datetimestamp, 'CSPOMS', 'CSPOMS', 'CSPOMS') for row in extracted_data])

        # Commit the changes
        print("Data inserted successfully.")
        # Commit the changes
        post_conn.commit()

        print("Data commited successfully.")

    except Exception as e:
        print(f"Error inserting data to table: {table_name}. Exception: {e}")
#         post_cur.rollback()

# Example usage
text_data_path = 'D:\\W\\RELEX\\RELEX\\Label Flag\\AWIROBLABEL.IN645.2023022804100111'
table_name = 'CSPOMS.XXPO_LABEL_FLAG_AWI_TRICEPS_STG'

# Column names in the same order as your data fields
column_names = ['FACILITY', 'WARE_HOUSE', 'RECORD_TYPE', 'FILLER', 'PO_NUMBER', 'APPT_CENTURY', 'APPT_YEAR',
                'APPT_MONTH', 'APPT_DATE', 'APPT_TIME_HH', 'APPT_TIME_MIN', 'PALLET_LABEL_PRINT',
                'PARENT_FACILITY', 'PO_RE_ROUTE_FLAG', 'TO_FACILITY', 'TO_WAREHOUSE', 'FROM_FACILITY',
                'FROM_WAREHOUSE', 'RECEIPT_NUMBER', 'APPT_CANCEL_FLAG', 'WHSE_COMMENTS', 'WHSE_COMMENTS2',
                'CARRIER', 'TRANSPORT_TYPE', 'RECEIPT_SLOT', 'APPOINTMENT_ID', 'DEPARTURE_DATE', 'DEPARTURE_TIME',
                'LABEL_DEL_EXAMPT_FLAG', 'FILLER1']

# # Extract data from text file
# extracted_data = process_text_data(text_data_path)

# Insert data into the table
insert_data_to_table(result, table_name, post_cur, column_names,request_id)


[(0, 2), (2, 4), (4, 6), (6, 8), (8, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 27), (27, 29), (29, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 47), (47, 48), (48, 78), (78, 138), (138, 144), (144, 145), (145, 152), (152, 163), (163, 171), (171, 175), (175, 176), (176, 240)]
[('01', '01', 'AP', '00', '197306', '20', '23', '02', '27', '07', '30', 'Y', '01', '', '', '', '', '', '000085545', 'N', '', 'S/W TO FOLLOW AT LEADTIME', '001178', 'T', '097', '', '', '', '', ''), ('01', '01', 'AP', '00', '198572', '20', '23', '02', '28', '03', '30', 'Y', '01', '', '', '', '', '', '000085541', 'N', '', 'S/W 94198572,9419881          SHIPPER 4/3 S/W 94198572,941', '999999', 'B', '101', '', '', '', '', ''), ('01', '01', 'AP', '00', '198881', '20', '23', '02', '28', '03', '30', 'Y', '01', '', '', '', '', '', '000085540', 'N', '', 'S/W 94198572,9419881', '999999', 'B', '101', '', '', '', '', ''), ('01', '02', 'AP', '00', '198910', '20', '23', '02', '28', '03', '30', 'Y'

Request ID: 1
INSERT INTO CSPOMS.XXPO_LABEL_FLAG_AWI_TRICEPS_STG (request_id, FACILITY, WARE_HOUSE, RECORD_TYPE, FILLER, PO_NUMBER, APPT_CENTURY, APPT_YEAR, APPT_MONTH, APPT_DATE, APPT_TIME_HH, APPT_TIME_MIN, PALLET_LABEL_PRINT, PARENT_FACILITY, PO_RE_ROUTE_FLAG, TO_FACILITY, TO_WAREHOUSE, FROM_FACILITY, FROM_WAREHOUSE, RECEIPT_NUMBER, APPT_CANCEL_FLAG, WHSE_COMMENTS, WHSE_COMMENTS2, CARRIER, TRANSPORT_TYPE, RECEIPT_SLOT, APPOINTMENT_ID, DEPARTURE_DATE, DEPARTURE_TIME, LABEL_DEL_EXAMPT_FLAG, FILLER1, creation_date, last_update_date, created_by, last_updated_by, last_update_login) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
Data inserted successfully.
Data commited successfully.


In [10]:
positions

[(0, 2),
 (2, 4),
 (4, 6),
 (6, 8),
 (8, 14),
 (14, 16),
 (16, 18),
 (18, 20),
 (20, 22),
 (22, 24),
 (24, 26),
 (26, 27),
 (27, 29),
 (29, 30),
 (30, 32),
 (32, 34),
 (34, 36),
 (36, 38),
 (38, 47),
 (47, 48),
 (48, 78),
 (78, 138),
 (138, 144),
 (144, 145),
 (145, 152),
 (152, 163),
 (163, 171),
 (171, 175),
 (175, 176),
 (176, 240)]